In [4]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
import numpy as np
import matplotlib.pyplot as plt
import os

os.makedirs("outputs", exist_ok=True)

simulator = AerSimulator()

# -------- Prepare Bell state |Ψ⁻> --------

def prepare_psi_minus():
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.x(1)
    qc.z(0)
    return qc

# -------- Measurement functions --------

def measure_basis(qc, basis_a, basis_b):
    """
    Add measurements in the specified bases.
    basis_a and basis_b are 'X', 'Z', W, V
    """
    c = qc.remove_final_measurements(inplace=False)
    
    # For qubit 0 (A)
    if basis_a == 'X':
        c.h(0)
    elif basis_a == 'Z':
        pass  # already Z
    
    # For qubit 1 (B)
    if basis_b == 'X':
        c.h(1)
    elif basis_b == 'Z':
        pass
    elif basis_b == W:
        c.ry(-np.pi/4, 1)
    elif basis_b == V:
        c.z(1)
        c.ry(-np.pi/4, 1)
    
    c.measure_all()
    return c

# Define the directions
W = (1/np.sqrt(2), 0, 1/np.sqrt(2))  # 1/√2 (X + Z)
V = (-1/np.sqrt(2), 0, 1/np.sqrt(2))  # 1/√2 (-X + Z)

measurements = {
    "X_W": ('X', W),
    "X_V": ('X', V),
    "Z_W": ('Z', W),
    "Z_V": ('Z', V)
}

# -------- Run measurements --------

shots = 1024

results = {}

for name, (basis_a, basis_b) in measurements.items():
    qc_psi = prepare_psi_minus()
    qc_meas = measure_basis(qc_psi, basis_a, basis_b)
    job = simulator.run(qc_meas, shots=shots)
    result = job.result()
    counts = result.get_counts(qc_meas)
    results[name] = counts
    print(f"{name}: {counts}")
    
    # Save histogram
    fig = plot_histogram(counts, title=name)
    fig.savefig(f"outputs/{name}_histogram.png")
    plt.show()

# -------- Calculate expectations --------

def calculate_expectation(counts, basis_a, basis_b):
    total = sum(counts.values())
    expectation = 0
    for outcome, count in counts.items():
        # outcome is 'ab 00' or similar, clean it
        outcome = outcome.replace(' ', '')
        a = outcome[0]
        b = outcome[1]
        x = 1 if a == '0' else -1
        y = 1 if b == '0' else -1
        expectation += (x * y) * (count / total)
    return expectation

expectations = {}
for name, counts in results.items():
    basis_a, basis_b = measurements[name]
    exp = calculate_expectation(counts, basis_a, basis_b)
    expectations[name] = exp
    print(f"<{name}> = {exp}")

# Calculate S
S = expectations["X_W"] - expectations["X_V"] + expectations["Z_W"] + expectations["Z_V"]
print(f"S = {S}")
print(f"|S| = {abs(S)}")

# -------- Task 2: Statistics --------

num_runs = 5
S_values = []

for run in range(num_runs):
    run_results = {}
    for name, (basis_a, basis_b) in measurements.items():
        qc_psi = prepare_psi_minus()
        qc_meas = measure_basis(qc_psi, basis_a, basis_b)
        job = simulator.run(qc_meas, shots=shots)
        result = job.result()
        counts = result.get_counts(qc_meas)
        run_results[name] = counts
    
    run_expectations = {}
    for name, counts in run_results.items():
        basis_a, basis_b = measurements[name]
        exp = calculate_expectation(counts, basis_a, basis_b)
        run_expectations[name] = exp
    
    S_run = run_expectations["X_W"] - run_expectations["X_V"] + run_expectations["Z_W"] + run_expectations["Z_V"]
    S_values.append(abs(S_run))
    print(f"Run {run+1}: |S| = {abs(S_run)}")

mean_S = np.mean(S_values)
std_S = np.std(S_values)
print(f"Mean |S| = {mean_S}")
print(f"Std |S| = {std_S}")

X_W: {'01 00': 417, '10 00': 455, '00 00': 85, '11 00': 67}
X_V: {'10 00': 81, '00 00': 431, '11 00': 449, '01 00': 63}
Z_W: {'00 00': 93, '10 00': 441, '01 00': 418, '11 00': 72}
Z_V: {'01 00': 450, '00 00': 89, '10 00': 411, '11 00': 74}
<X_W> = -0.703125
<X_V> = 0.71875
<Z_W> = -0.677734375
<Z_V> = -0.681640625
S = -2.78125
|S| = 2.78125
Run 1: |S| = 2.845703125
Run 2: |S| = 2.876953125
Run 3: |S| = 2.845703125
Run 4: |S| = 2.880859375
Run 5: |S| = 2.890625
Mean |S| = 2.86796875
Std |S| = 0.01871741961048584
